In [22]:
import yaml
from yaml import Loader

conf_path = '/Users/akshita/Documents/Research/Makin/Auditory_Cortex/conf/ridge_conf.yaml'
with open(conf_path, "r") as f:
    manifest = yaml.load(f, Loader=Loader)

data_path = manifest['data_path']
subject = manifest['sub']


In [23]:
manifest

{'data_path': '/depot/jgmakin/data/auditory_cortex/josh_data/data',
 'sub': '200206',
 'output_dir': '/scratch/gilbreth/akamsali/Research/Makin/outputs/neuron_corr/ridge_regression'}

In [1]:
from auditory_cortex.Regression import transformer_regression

import numpy as np

reg = transformer_regression("/Users/akshita/Documents/Research/Makin/data", '200206')

In [15]:
from sklearn.model_selection import KFold
import time
from auditory_cortex.ridge_regression import RidgeRegression 

r2t = 0
r2v = 0
r2tt = 0
sp = 1
l = 0
w = 80

test_list = np.arange(450,499).tolist()
train_val_list = np.arange(1,450)


start = time.time()
k_val_test , def_w_test, offset_test, z_vals_test = reg.get_layer_values(l, win=w, sent_list=test_list)
print(f"layer extract time = {time.time() - start}")
start = time.time()
n_vals_test = reg.get_all_channels(def_w=def_w_test, offset=offset_test, k_val=k_val_test, sent_list=test_list)
print(f"spike extract time = {time.time() - start}")


kf = KFold(n_splits=5, shuffle=True)
# kf.get_n_splits(train_val_list)

# for alpha in 
ridge_model = RidgeRegression(alpha=10)
for train, val in kf.split(train_val_list):
    start = time.time()
    
    k_val_train , def_w_train, offset_train, z_vals_train = reg.get_layer_values(layer=l, win=w, sent_list=train_val_list[train])
    n_vals_train = reg.get_all_channels(def_w=def_w_train, offset=offset_train, k_val=k_val_train, sent_list=train_val_list[train])

    k_val_val , def_w_val, offset_val, z_vals_val = reg.get_layer_values(layer=l,win=w, sent_list=train_val_list[val])
    n_vals_val = reg.get_all_channels(def_w=def_w_val, offset=offset_val, k_val=k_val_val, sent_list=train_val_list[val])

    ridge_model.fit(z_vals_train, n_vals_train)
    n_hat_train = ridge_model.predict(z_vals_train)
    n_hat_val = ridge_model.predict(z_vals_val)

    n_hat_test = ridge_model.predict(z_vals_test)
    r2t += reg.cc_norm(n_hat_train, n_vals_train, sp=sp)
    r2v += reg.cc_norm(n_hat_val, n_vals_val, sp=sp)
    r2tt += reg.cc_norm(n_hat_test, n_vals_test, sp=sp)
    print(f"one iter time = {time.time() - start}, {[r2t, r2v, r2tt]}")


layer extract time = 9.90459394454956
spike extract time = 50.53598380088806
(1025, 1025)
one iter time = 620.7318079471588, [0.024613460858018882, 0.04920637637034571, 0.23183328757946922]
(1025, 1025)
one iter time = 857.3485221862793, [0.10176304131394884, 0.12093131105970505, 0.5003983855133335]
(1025, 1025)
one iter time = 878.3620851039886, [0.15076351402257215, 0.32361124245023265, 0.7716481880978996]
(1025, 1025)
one iter time = 851.9852223396301, [0.23663748945747792, 0.23116299459214198, 1.0368130291887059]
(1025, 1025)
one iter time = 849.7227997779846, [0.2939962672312589, 0.3555912918242024, 1.2482885918871236]


In [1]:
from auditory_cortex.Regression import transformer_regression

import json 
import sys

# print(sys.argv)
# sub = sys.argv[1]
# w = sys.argv[2]

# print("arg vals",sub, int(w))
sub = '200206'
w = 80
reg = transformer_regression('/depot/jgmakin/data/auditory_cortex/josh_data/data',sub)

# channels = np.arange(0,reg.dataset.num_channels).tolist()
num_layers = len(reg.layers)
corr_values = {}
for ch in range(1):
    print(ch)
    R2t =[]  
    R2v =[] 
    R2tt =[]
    for l in range(num_layers):
        r2t, r2v,r2tt = reg.get_cc_norm(l,int(w),channel=ch, delay=0)
        # print(r2t)
        R2t.append(r2t.item())
        R2v.append(r2v.item())
        R2tt.append(r2tt.item())
        # PCt.append(pct)
        # PCv.append(pcv)
        # PCtt.append(pctt)
    corr_values[ch] =  {"train": R2t, "val": R2v, "test": R2tt}
    print(corr_values[ch])
# fig, ax = plt.subplots(1,2, figsize=(14,6), sharey=True)

# with open("/scratch/gilbreth/akamsali/Research/Makin/outputs/neuron_corr/"+sub + "_" + str(w), 'w') as f:
#     json.dump(corr_values, f)

with open("/scratch/gilbreth/akamsali/Research/Makin/Auditory_Cortex/"+sub + "_" + str(w), 'a') as f:
    json.dump(corr_values, f)

    
# plt.plot(R2t, label='Training')
# plt.plot(R2v, label='Val')
# plt.plot(R2tt, label='Test')
# plt.legend()


# ax[0].plot(R2t, label='Training')
# ax[0].plot(R2v, label='Val')
# ax[0].plot(R2tt, label='Test')
# ax[0].legend()
# ax[0].set_title("R2")


# ax[1].plot(PCt, label='Training')
# ax[1].plot(PCv, label='Val')
# ax[1].plot(PCtt, label='Test')
# ax[1].legend()
# ax[1].set_title("PC")

# plt.title("200213, bin_width="+ str(w))
# # plt.savefig("/Users/akshita/Documents/Research/Makin/outputs/200213_" + str(w))
# plt.show()

/home/akamsali/anaconda3/envs/research_env/lib/python3.8/site-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  /opt/conda/conda-bld/pytorch_1631630815121/work/aten/src/ATen/native/BinaryOps.cpp:467.)
  return torch.floor_divide(self, other)


0
{'train': [0.3218077348683456, 0.3899081205000163, 0.35454461371827206, 0.3543961330861095, 0.3445456526471164, 0.32071031788373433, 0.3184274042862006, 0.320425945454863, 0.3030256070758387, 0.2934304631292377, 0.2851580342688245, 0.27108428531378215], 'val': [0.2056477240942051, 0.24880887053054063, 0.25082439133975687, 0.25722088504772755, 0.24108598195182487, 0.20623103219882505, 0.20336928317963507, 0.2245079969803741, 0.18140871890469296, 0.16967012435263001, 0.15788541065265255, 0.1257959259611648], 'test': [0.14655752190020152, 0.185419083508866, 0.2615541987084618, 0.2107761655510611, 0.20502966576455223, 0.1722682160663232, 0.17428930390233013, 0.1835482748473299, 0.19003016359664793, 0.16813908876063358, 0.1514884329554947, 0.14721644348375587]}


In [2]:
corr_values

{0: {'train': [0.3218077348683456,
   0.3899081205000163,
   0.35454461371827206,
   0.3543961330861095,
   0.3445456526471164,
   0.32071031788373433,
   0.3184274042862006,
   0.320425945454863,
   0.3030256070758387,
   0.2934304631292377,
   0.2851580342688245,
   0.27108428531378215],
  'val': [0.2056477240942051,
   0.24880887053054063,
   0.25082439133975687,
   0.25722088504772755,
   0.24108598195182487,
   0.20623103219882505,
   0.20336928317963507,
   0.2245079969803741,
   0.18140871890469296,
   0.16967012435263001,
   0.15788541065265255,
   0.1257959259611648],
  'test': [0.14655752190020152,
   0.185419083508866,
   0.2615541987084618,
   0.2107761655510611,
   0.20502966576455223,
   0.1722682160663232,
   0.17428930390233013,
   0.1835482748473299,
   0.19003016359664793,
   0.16813908876063358,
   0.1514884329554947,
   0.14721644348375587]}}

In [ ]:
import numpy as np

from Dataset import Neural_Data


neural_data = Neural_Data('/Users/akshita/Documents/Research/Makin/data',"200206")

In [ ]:
repeated_sents = {}
for i in range(499):
    data = neural_data.retrieve_spikes_count_for_all_trials(i, w=100)
    if data[1].shape[0] > 1:
        repeated_sents[i] = data[1].shape
print(repeated_sents)

In [ ]:
from scipy import stats
import matplotlib.pyplot as plt
import csv
# print(channel.shape)

# sents = [12,13,32,43,56,163,212,218,287,308]
sents = [12]
# sent_vals = {}
# wins = [1,5,10,20,30,40,50,75,100,200]
wins = [10]
for w in wins:
    for sent in sents:
        data = neural_data.retrieve_spikes_count_for_all_trials(sent, w=w)

        for j in range(len(data)):
            channel = data[j]
            # print(channel.shape)
            cnt = 0
            val = []
            p_r = []
            for shift in [0]:#1, 5, 10]:
                for i in range(11):
                    cnt+=1.0
                    ind = np.arange(0,11,1)
                    b_ind = ind[i]
                    a_ind = np.delete(ind, i)
                    # print(a_ind, b_ind)
                    a = np.mean(channel[a_ind, :], axis=0)
                    b = channel[b_ind,:]
                    b = np.roll(b, shift)
                    
                    res = stats.linregress(a, b)
                    pearson_r = stats.pearsonr(a, b)
                    val.append(res.rvalue**2)
                    p_r.append(pearson_r[0]**2)

                with open("/home/amy/Documents/Research/Makin/outputs/good_sessions/linreg_cs_win_" + str(w)+ "_"+ str(sent) +"_" + str(shift)+".csv", 'a') as f:
                    writer=csv.writer(f)
                    row = val 
                    writer.writerow(row)
                    f.close()
    print("done: ", w)


In [ ]:
ind = np.arange(0,11,1)
b_ind = ind[i]
a_ind = np.delete(ind, i)
print(a_ind, b_ind)

In [ ]:
import pandas as pd
fig, ax = plt.subplots(4, len(wins),sharey=True,figsize=(5,6))

for i, shift in enumerate([0, 1, 5, 10]):
    for j, w in enumerate(wins):
        for sent in sents:
        # plt.figure()
            d = pd.read_csv("/home/amy/Documents/Research/Makin/outputs/good_sessions/linreg_cs_win_" + str(w)+ "_"+ str(sent) +"_" + str(shift)+ ".csv", header=None)
            d.columns = ["channels", "reg_r2", "pearson_r2"]

            avg_r2 = d['reg_r2'].to_numpy()
            pearson_r2 = d['pearson_r2'].to_numpy()
            # print(avg_r2.shape, pearson_r2.shape)

            # channels = d['channels'].to_numpy()
            # plt.figure()
            # ax[i, j].plot(channels, avg_r2, label=str(sent))
            ax[i].plot(avg_r2-pearson_r2, label=str(sent))
            ax[0].set_title(f"Win={w}")
            ax[i].set_ylabel(f"Shift={shift}")
            plt.ylim(-1.25, 0.8)
        # plt.xlabel("channels")
    # plt.title(f"Win = {w}" , fontsize=16)plt.legend()
        # plt.savefig("/home/amy/Documents/Research/Makin/outputs/correlation/linreg_cs_win_" + str(w)+ ".png")
plt.show()

In [ ]:
for j, w in enumerate(wins):
    for i, shift in enumerate([0,1, 5, 10]):
        plt.figure()
        for sent in sents:
            d = pd.read_csv("/home/amy/Documents/Research/Makin/outputs/correlation/linreg_cs_win_" + str(w)+ "_"+ str(sent) +"_" + str(shift)+ ".csv", header=None)
            d.columns = ["channels", "reg_r2", "pearson_r2"]

            avg_r2 = d['reg_r2'].to_list()
            pearson_r2 = d['pearson_r2'].to_list()
            channels = d['channels'].to_list()
            # plt.figure()
            plt.plot(channels, avg_r2, label=str(sent))
            plt.title(f"Win={w}, Shift={shift}")
            # ax[i,j].set_ylabel(f"Shift={shift}")
            plt.ylim(-0.5, 0.4)
        # plt.xlabel("channels")
    # plt.title(f"Win = {w}" , fontsize=16)plt.legend()
        # plt.savefig("/home/amy/Documents/Research/Makin/outputs/correlation/linreg_cs_win_" + str(w)+ ".png")
        plt.show()

In [ ]:
import pandas as pd

for sent in sents:
    d2 = pd.read_csv("/home/amy/Documents/Research/Makin/outputs/correlation/linreg_loo_win_ms10_"+ str(sent) +".csv", header=None)
    d2.columns = ["channels", "avg_r2"]

    avg_R2 = d2['avg_r2'].to_list()
    channels = d2['channels'].to_list()
    
    plt.plot(channels, avg_R2 ,label=str(sent))
    plt.ylim(-1.25, 0.8)
plt.title("Correlation for sentences ms vs channels, w=10", fontsize=16)
plt.legend()
plt.show()

In [ ]:
import numpy as np

a = np.zeros(100)
b = np.zeros(100)
ind_a = np.random.randint(0,100,10)
ind_b = np.random.randint(0,100,10)

a[ind_a] = 1    
b[ind_b] = 1
res = stats.linregress(a,b)

res.rvalue**2

In [ ]:
for w in [1]:
    for sent in [12]:
        data = neural_data.retrieve_spikes_count_for_all_trials(sent, w=w)

        for j in [20]:
            channel = data[j]
            # for shift in [1, 5, 10]:
            for i in range(1):
                plt.plot(channel[i], label=j)

    plt.legend()
    plt.show()